In [8]:
# -*- coding: utf-8 -*-
"""
Created on Thu Sep 22 11:00:17 2022
@author: David
"""

#######################################################################

#Import all of the wonderful things we like and need to manage data and make plots.
import numpy as np
import matplotlib.pyplot as plt
import pandas as pds
import pathlib 
import math
import cmath
from scipy import stats
from scipy.constants import k
from copy import deepcopy

#######################################################################
config01 = [(168, 15), (493, 72), (22, 299), (466, 75), (360, 263)]
config00 = [(168, 15), (493, 72), (22, 299), (466, 75)]
config_idx = [0,1,2,3,4]
D_mat = np.zeros((len(config00), len(config00)))
for i in range(len(config00)):
    for k in range(len(config00)):
        if i !=k:
            D_mat[i][k] = np.sqrt((config00[i][0]-config00[k][0])**2+(config00[i][1]-config00[k][1])**2)

In [9]:
D_mat

array([[  0.        , 329.96060371, 319.33054974, 303.98026252],
       [329.96060371,   0.        , 522.84797026,  27.16615541],
       [319.33054974, 522.84797026,   0.        , 497.30473555],
       [303.98026252,  27.16615541, 497.30473555,   0.        ]])

In [6]:


#Define a function to caluclate the route length:
def calc_route_length(config):
    
    route_length = 0
    
    #For each city, calculate the distance between it and previous.
    for i in range(len(config)):
        d = np.sqrt((config[i][0] - config[i-1][0])**2 + (config[i][1] - config[i-1][1])**2)
        route_length = route_length + d
        
    return route_length



#Define a function to generate a trial configuration where we swap the order of visiting two cities.
def generate_trial_config(config):
    
    #Make a deep copy so we don't accidentally mess up our config:
    trial_config = deepcopy(config)
    
    #Generate one random city to switch:
    first_swap = np.random.randint(0, len(config))
    #Assign this value to our second swap:
    second_swap = first_swap
    
    #Generate a second random city to switch, making sure it's different from our first:
    while second_swap == first_swap:
        second_swap = np.random.randint(0, len(config))
    
    #Now swap our indices:
    hold_value = deepcopy(trial_config[first_swap])
    trial_config[first_swap] = trial_config[second_swap]
    trial_config[second_swap] = hold_value
    
    #print(trial_config)
    
    return trial_config, first_swap, second_swap

#Define a function to calculate the change in distance from our configuration change:
def calc_distance_change(config, trial_config, first_swap, second_swap):
    
    #Calculate original distance:
    x_i = config[first_swap][0]
    y_i = config[first_swap][1]
    #Distance between first swap -1:
    x_1_min = config[first_swap-1][0]
    y_1_min = config[first_swap-1][1]
    #Pythagorean theorem:
    d_one_min = np.sqrt((x_i - x_1_min)**2 + (y_i - y_1_min)**2)
    #Distance between first swap +1, set our bounds on our array of values:
    if (first_swap + 1) > (len(config)-1):
        swap_max = 0
    else:
        swap_max = first_swap +1
    x_1_max = config[swap_max][0]
    y_1_max = config[swap_max][1]
    #Pythagoran theorem:
    d_one_max = np.sqrt((x_1_max - x_i)**2 + (y_1_max - y_i)**2)
    #Distance between second swap -1:
    x_j = config[second_swap][0]
    y_j = config[second_swap][1]
    x_2_min = config[second_swap-1][0]
    y_2_min = config[second_swap-1][1]
    #Pythagorean theorem:
    d_two_min = np.sqrt((x_j - x_2_min)**2 + (y_j - y_2_min)**2)
    #Distance between second swap +1, set our bounds on array of values:
    if (second_swap + 1) > (len(config)-1):
        swap_max = 0
    else:
        swap_max = second_swap +1
    x_2_max = config[swap_max][0]
    y_2_max = config[swap_max][1]
    #Pythagorean theorem:
    d_two_max = np.sqrt((x_2_max - x_j)**2 + (y_2_max - y_j)**2)
    
    #Now we have our original distance that navigating to these points took:
    original_distance = d_one_min + d_one_max + d_two_min + d_two_max
    #print(original_distance)
    
    #Now repeat for the trial configuration...
    #Calculate original distance:
    x_i = trial_config[first_swap][0]
    y_i = trial_config[first_swap][1]
    #Distance between first swap -1:
    x_1_min = trial_config[first_swap-1][0]
    y_1_min = trial_config[first_swap-1][1]
    #Pythagorean theorem:
    d_one_min = np.sqrt((x_i - x_1_min)**2 + (y_i - y_1_min)**2)
    #Distance between first swap +1, set our bounds on our array of values:
    if (first_swap + 1) > (len(trial_config)-1):
        swap_max = 0
    else:
        swap_max = first_swap +1
    x_1_max = trial_config[swap_max][0]
    y_1_max = trial_config[swap_max][1]
    #Pythagoran theorem:
    d_one_max = np.sqrt((x_1_max - x_i)**2 + (y_1_max - y_i)**2)
    #Distance between second swap -1:
    x_j = trial_config[second_swap][0]
    y_j = trial_config[second_swap][1]
    x_2_min = trial_config[second_swap-1][0]
    y_2_min = trial_config[second_swap-1][1]
    #Pythagorean theorem:
    d_two_min = np.sqrt((x_j - x_2_min)**2 + (y_j - y_2_min)**2)
    #Distance between second swap +1, set our bounds on array of values:
    if (second_swap + 1) > (len(config)-1):
        swap_max = 0
    else:
        swap_max = second_swap +1
    x_2_max = trial_config[swap_max][0]
    y_2_max = trial_config[swap_max][1]
    #Pythagorean theorem:
    d_two_max = np.sqrt((x_2_max - x_j)**2 + (y_2_max - y_j)**2)
    
    #Now we have our new distance that navigating to these points took:
    new_distance = d_one_min + d_one_max + d_two_min + d_two_max
    #print(new_distance)
    
    #Calculate our change in distance:
    delta_d = new_distance - original_distance
    #print(delta_d)
    
    return delta_d

#Define a function to calculate our probability and evaluate it:
def evaluate_route_change(delta_d, T):
    
    #Calculate our probability:
    P = math.exp((-1*delta_d)/(T))
    #Geneate uniform random variable:
    u = np.random.uniform()
    #Estimate whether we're keeping this or not:
    if u <= P:
        accept = True
    else:
        accept = False
    
    return accept
    
#Define a function to minimize our route:
def minimize_route(config, T, roc_tol, mean_samples):
    
    #Create counter for our number of steps, and initial large value for rate of change:
    numsteps = 0
    roc = 10*roc_tol
    
    #Find our initial route length - we'll need this later...
    route_length = calc_route_length(config)
    #Create array to store new route lengths:
    route_lengths = []
    
    #Want rate of change of our config over some number of steps to be small...
    while roc > roc_tol:
        
        #Generate trial configuration and calulate change in distance:
        trial_config, first_swap, second_swap = generate_trial_config(config)
        delta_d = calc_distance_change(config, trial_config, first_swap, second_swap)
        
        #Evaluate if we are keeping our new configuration or not...
        if delta_d <= 0:
            config = deepcopy(trial_config)
            route_length = route_length + delta_d
            route_lengths.append(route_length)
            #Increment numsteps counter:
            numsteps = numsteps +1
        else:
            accept = evaluate_route_change(delta_d, T)
            if accept == True:
                config = deepcopy(trial_config)
                route_length = route_length + delta_d
                route_lengths.append(route_length)
                #Increment numsteps counter:
                numsteps = numsteps +1
            else: 
                pass
        
        #If we have more than 100 steps, evaluate our mean route length over the last 50 steps,
        #compare to the 50 before, and see what our rate of change is.
        if numsteps > (mean_samples*2):
            new_mean = np.mean(route_lengths[numsteps-mean_samples:])
            old_mean = np.mean(route_lengths[numsteps-(mean_samples*2):numsteps-mean_samples])
            roc = abs(new_mean - old_mean)
            
    return config, route_lengths

#Define a function to compute our smallest route overall:
def travelling_salesman(N, M, initial_T, delta_T, roc_tol_at_T, roc_tol, mean_samples, failsafe):
    
    #Generate initial configuration:
    initial_config = config0
    #print(initial_config)
    #Create equivalent of do-while loop:
    while True:
        #Do some math with our original route, including calculating route lengths etc.
        config = initial_config
        old_route_length = calc_route_length(config)
        print(old_route_length)
        old_route_lengths = []
        old_route_lengths.append(old_route_length)
        #Create set of unique routes.
        unique_routes = set(np.around(old_route_lengths,3))
        #Set an initial roc we can evaluate against.
        roc = 10*roc_tol
        #Set our initial T.
        T = initial_T
        
        #Run while our roc is greater than our tol, and our temperature is above 0:
        while (roc > roc_tol) and (T > 0):
            #Minimize route at current T:
            config, route_lengths = minimize_route(config, T, roc_tol_at_T, mean_samples)
            #If we've come up with a shorter route:
            if len(route_lengths) != 0:
                #Calculate change in mean:
                old_mean = np.mean(old_route_lengths)
                new_mean = np.mean(route_lengths)
                roc = abs(new_mean - old_mean)
                #If our roc is still greater:
                if roc > roc_tol:
                    #Swap over and update our recent routes...
                    old_route_lengths = deepcopy(route_lengths)
                    temp_set_routes = set(np.around(old_route_lengths,3))
                    unique_routes = unique_routes.union(temp_set_routes)
                else:
                    pass
            else:
                pass
            
            #Increment T downwards...
            T = T - delta_T
            print(T)
            
            #If T goes below zero...
            if T <= 0:
                route_lengths.append(old_route_lengths[-1])
            else:
                pass
        
        #Sort our set of routes so we can use these in failsafe.
        sorted_routes = sorted(unique_routes)
        #print(sorted_routes)
            
        #If we're not using the failsafe, or if our route is as short as the shortest possible route after hitting roc tolerance, break the loop.
        if (failsafe == False) or (round(route_lengths[-1],3) <= (round(sorted_routes[0],3)+0.001)):
            break
    
    return config, unique_routes, route_lengths[-1]

#######################################################################
#Main: let's run some functions:

#config, unique_routes, final_route_length = travelling_salesman(6, 10, 20, 0.05, 0.001, 0.001, 100, True)
#print('Set of Unique Route Lengths: ' + str(unique_routes))
#print('Final Route Length: ' + str(final_route_length))

config, unique_routes, final_route_length = travelling_salesman(6, 500, 25, 0.5, 0.1, 0.1, 100, False)
print(config)
print('Final Route Length: ' + str(final_route_length))

1879.574044428653
24.5
24.0
23.5
23.0
[(168, 15), (22, 299), (360, 263), (466, 75), (493, 72)]
Final Route Length: 1232.1930644937024


In [12]:
def calc_route_length(config, D_mat):
    
    route_length = 0
    #print(D_mat)
    #For each city, calculate the distance between it and previous.
    for i in range(len(config)):
        start_idx = config[i-1]
        end_idx = config[i]
        #print(start_idx,end_idx)
        
        d = D_mat[start_idx][end_idx]
        route_length = route_length + d
        
    return route_length

#Define a function to generate a trial configuration where we swap the order of visiting two cities.
def generate_trial_config(config):
    
    #Make a deep copy so we don't accidentally mess up our config:
    trial_config = deepcopy(config)
    
    #Generate one random city to switch:
    first_swap = np.random.randint(0, len(config))
    #Assign this value to our second swap:
    second_swap = first_swap
    
    #Generate a second random city to switch, making sure it's different from our first:
    while second_swap == first_swap:
        second_swap = np.random.randint(0, len(config))
    
    #Now swap our indices:
    hold_value = deepcopy(trial_config[first_swap])
    trial_config[first_swap] = trial_config[second_swap]
    trial_config[second_swap] = hold_value
    
    #print(trial_config)
    
    return trial_config, first_swap, second_swap

def calc_distance_change(config, trial_config, first_swap, second_swap, D_mat):
    
    #Calculate original distance:
    idx_i = config[first_swap]
    #print(idx_i)
    #Distance between first swap -1:
    idx_1_min = config[first_swap-1]
   
    #Pythagorean theorem:
    
    d_one_min = D_mat[idx_i][idx_1_min]
    #Distance between first swap +1, set our bounds on our array of values:
    if (first_swap + 1) > (len(config)-1):
        swap_max = 0
    else:
        swap_max = first_swap +1
    idx_1_max = config[swap_max]
    
    #Pythagoran theorem:
    d_one_max = D_mat[idx_1_max][idx_i]
    #Distance between second swap -1:
    idx_j = config[second_swap]
    idx_2_min = config[second_swap-1]
    
    #Pythagorean theorem:
    
    d_two_min = D_mat[idx_j][idx_2_min]
    #Distance between second swap +1, set our bounds on array of values:
    if (second_swap + 1) > (len(config)-1):
        swap_max = 0
    else:
        swap_max = second_swap +1
        
    idx_2_max = config[swap_max]
    
    #Pythagorean theorem:
    d_two_max = D_mat[idx_2_max][idx_j]
    
    #Now we have our original distance that navigating to these points took:
    original_distance = d_one_min + d_one_max + d_two_min + d_two_max
    #print(original_distance)
    
    #Now repeat for the trial configuration...
    #Calculate original distance:
    idx_i = trial_config[first_swap]
    
    #Distance between first swap -1:
    idx_1_min =  trial_config[first_swap-1]
   
    #Pythagorean theorem:
    d_one_min = D_mat[idx_i][idx_1_min]
    #Distance between first swap +1, set our bounds on our array of values:
    if (first_swap + 1) > (len(trial_config)-1):
        swap_max = 0
    else:
        swap_max = first_swap +1
    idx_1_max = trial_config[swap_max]
    
    #Pythagoran theorem:
    
    d_one_max = D_mat[idx_1_max][idx_i]
    
    #Distance between second swap -1:
    idx_j = trial_config[second_swap]
    idx_2_min = trial_config[second_swap-1]
    
    #Pythagorean theorem:
   
    d_two_min = D_mat[idx_j][idx_2_min]
    #Distance between second swap +1, set our bounds on array of values:
    if (second_swap + 1) > (len(config)-1):
        swap_max = 0
    else:
        swap_max = second_swap +1
    idx_2_max = trial_config[swap_max]
    
    #Pythagorean theorem:
    d_two_max = D_mat[idx_2_max][idx_j]
    #Now we have our new distance that navigating to these points took:
    new_distance = d_one_min + d_one_max + d_two_min + d_two_max
    #print(new_distance)
    
    #Calculate our change in distance:
    delta_d = new_distance - original_distance
    #print(delta_d)
    
    return delta_d

#Define a function to calculate our probability and evaluate it:
def evaluate_route_change(delta_d, T):
    
    #Calculate our probability:
    P = math.exp((-1*delta_d)/(T))
    #Geneate uniform random variable:
    u = np.random.uniform()
    #Estimate whether we're keeping this or not:
    if u <= P:
        accept = True
    else:
        accept = False
    
    return accept


def minimize_route(config, T, roc_tol, mean_samples, D_mat):
    
    #Create counter for our number of steps, and initial large value for rate of change:
    numsteps = 0
    roc = 10*roc_tol
    
    #Find our initial route length - we'll need this later...
    route_length = calc_route_length(config, D_mat)
    #Create array to store new route lengths:
    route_lengths = []
    
    #Want rate of change of our config over some number of steps to be small...
    while roc > roc_tol:
        
        #Generate trial configuration and calulate change in distance:
        trial_config, first_swap, second_swap = generate_trial_config(config)
        delta_d = calc_distance_change(config, trial_config, first_swap, second_swap, D_mat)
        
        #Evaluate if we are keeping our new configuration or not...
        if delta_d <= 0:
            config = deepcopy(trial_config)
            route_length = route_length + delta_d
            route_lengths.append(route_length)
            #Increment numsteps counter:
            numsteps = numsteps +1
        else:
            accept = evaluate_route_change(delta_d, T)
            if accept == True:
                config = deepcopy(trial_config)
                route_length = route_length + delta_d
                route_lengths.append(route_length)
                #Increment numsteps counter:
                numsteps = numsteps +1
            else: 
                pass
        
        #If we have more than 100 steps, evaluate our mean route length over the last 50 steps,
        #compare to the 50 before, and see what our rate of change is.
        if numsteps > (mean_samples*2):
            new_mean = np.mean(route_lengths[numsteps-mean_samples:])
            old_mean = np.mean(route_lengths[numsteps-(mean_samples*2):numsteps-mean_samples])
            roc = abs(new_mean - old_mean)
            
    return config, route_lengths



def travelling_salesman(initial_config, initial_T, delta_T, roc_tol_at_T, roc_tol, mean_samples, failsafe, D_mat):
    #print(D_mat)
    #Generate initial configuration:
    #initial_config = config_idx
    #print(initial_config)
    #Create equivalent of do-while loop:
    while True:
        #Do some math with our original route, including calculating route lengths etc.
        config = initial_config
        old_route_length = calc_route_length(config, D_mat)
        
        old_route_lengths = []
        old_route_lengths.append(old_route_length)
        #Create set of unique routes.
        unique_routes = set(np.around(old_route_lengths,3))
        #Set an initial roc we can evaluate against.
        roc = 10*roc_tol
        #Set our initial T.
        T = initial_T
        
        #Run while our roc is greater than our tol, and our temperature is above 0:
        while (roc > roc_tol) and (T > 0):
            #Minimize route at current T:
            config, route_lengths =  minimize_route(config, T, roc_tol, mean_samples, D_mat)
            #If we've come up with a shorter route:
            if len(route_lengths) != 0:
                #Calculate change in mean:
                old_mean = np.mean(old_route_lengths)
                new_mean = np.mean(route_lengths)
                roc = abs(new_mean - old_mean)
                #If our roc is still greater:
                if roc > roc_tol:
                    #Swap over and update our recent routes...
                    old_route_lengths = deepcopy(route_lengths)
                    temp_set_routes = set(np.around(old_route_lengths,3))
                    unique_routes = unique_routes.union(temp_set_routes)
                else:
                    pass
            else:
                pass
            
            #Increment T downwards...
            T = T - delta_T
            print(T)
            
            #If T goes below zero...
            if T <= 0:
                route_lengths.append(old_route_lengths[-1])
            else:
                pass
        
        #Sort our set of routes so we can use these in failsafe.
        sorted_routes = sorted(unique_routes)
        #print(sorted_routes)
            
        #If we're not using the failsafe, or if our route is as short as the shortest possible route after hitting roc tolerance, break the loop.
        if (failsafe == False) or (round(route_lengths[-1],3) <= (round(sorted_routes[0],3)+0.001)):
            break
    
    return config, unique_routes, route_lengths[-1]




In [13]:
#params from google maps api
# D_mat = np.array([[  0, 705, 388, 401, 260],
#        [757,   0, 691, 911, 716],
#        [335, 678,   0, 557, 252],
#        [408, 881, 528,   0, 368],
#        [281, 785, 307, 503,   0]])

D_mat = np.array([[  0, 705, 388, 401, 260],
       [705,   0, 691, 911, 716],
       [388, 691,   0, 557, 252],
       [401, 911, 557,   0, 368],
       [260, 785, 252, 368,   0]])
start_config =[0,1,2,3,4]

initial_config = start_config
initial_T = 25
delta_T = 0.5
roc_tol_at_T = 0.1
roc_tol = 0.1
mean_samples = 100
failsafe = False

config, unique_routes, final_route_length =travelling_salesman(initial_config, initial_T, delta_T, roc_tol_at_T, roc_tol, mean_samples, failsafe, D_mat)
print(config)
print(final_route_length)

24.5
24.0
23.5
23.0
22.5
22.0
21.5
21.0
20.5
20.0
19.5
19.0
18.5
18.0
17.5
[3, 4, 2, 0, 1]
2624
